<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Fitbit_V02_Classification_2023_08_28_6MWT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

!pip install ipywidgets
!pip install IPython

import ipywidgets as widgets
from IPython.display import display
import seaborn as sns
import sklearn.metrics as sm


def prim_plot(y_actual,y_pred):
  pred_y_df=pd.DataFrame(
      {
          "Actual_Value":y_actual,
          "Predicted_Value":y_pred,
          "Difference":y_actual-y_pred
      }
  )


  """sns.set(rc={'figure.figsize':(12,10)})
  scatter_poly2 = sns.lmplot(
      data = pred_y_df,
      x = "Actual_Value",
      y = "Predicted_Value",
      order = 2
  )"""

  sns.regplot(x="Actual_Value", y="Predicted_Value", data=pred_y_df)


def res(y_actual,y_pred):
  print("Mean absolute error =", round(sm.mean_absolute_error(y_actual, y_pred), 2))
  print("Mean squared error =", round(sm.mean_squared_error(y_actual, y_pred), 2))
  print("Median absolute error =", round(sm.median_absolute_error(y_actual, y_pred), 2))
  print("Explain variance score =", round(sm.explained_variance_score(y_actual, y_pred), 2))
  print("R2 score =", round(sm.r2_score(y_actual, y_pred), 2))


def sub_wise_splt(df,col,flage):
  total_count = len(set(df[col]))
  subjects=sorted(set(df[col]))

  train_axis = []
  test_axis = []

  count = 0

  if str(df[col].dtypes)=='object':
    for i in subjects:
      if count <= flage:
        train_axis.append(df.query(col+" == '"+i+"'"))
      else:
        test_axis.append(df.query(col+" == '"+i+"'"))
      count+=1
  else:
    for i in subjects:
      if count <= flage:
        train_axis.append(df.query(col+" == "+str(i)))
      else:
        test_axis.append(df.query(col+" == "+str(i)))
      count+=1


  return pd.concat(train_axis, axis=0).reset_index(drop = True),pd.concat(test_axis, axis=0).reset_index(drop = True)




from sklearn.preprocessing import LabelEncoder


def encode(data):
  cls = list(data)
  encoder=LabelEncoder()
  for i in cls:
    if str(data[i].dtypes) == 'object':
      data[i]=encoder.fit_transform(data[i])

  return data


def drop_str_col(data):
  cls = list(data)
  rmv=[]
  for i in cls:
    if str(data[i].dtypes) == 'object':
      rmv.append(i)
  data = data.drop(rmv, axis=1)
  return data




def items_check_box(items):
  new_keys=[]
  for i in items:
    i=widgets.Checkbox(
      value=False,
      description=str(i),
      disabled=False,
      indent=False
      )
    display(i)
    new_keys.append(i)
  return new_keys



def models_check_box(models):
  import ipywidgets as widgets
  from IPython.display import display
  new_keys=[]
  for i in models:
    i=widgets.Checkbox(
      value=False,
      description=str(i),
      disabled=False,
      indent=False
      )
    display(i)
    new_keys.append(i)
  return new_keys

result={}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.1 MB/s eta 0:00:00


In [54]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score,mean_squared_error
#dataset=pd.read_csv("C:/Users/Richard/Desktop/Fitbit/Fitbit_feature_merged_version_2023.06.20 - Main file.csv")
#dataset = pd.read_csv("/content/drive/MyDrive/Cornell/MADE Lab/Wearable Project/Data/Fitbit ML/Feature merge/Feature Matrix/Fitbit_feature_merged_version_2023.06.20 - Main file.csv")
#dataset=pd.read_excel("/content/drive/MyDrive/Cornell/MADE Lab/Wearable Project/Data/Fitbit ML/Feature merge/Fitbit_feature_merged_version_2023.06.29_SPO2_wo_missing column.xlsx")
#dataset = pd.read_csv("/content/drive/MyDrive/Cornell/MADE Lab/Wearable Project/Data/Fitbit ML/Feature merge/ML/Fitbit_feature_merged_version_RepeatedData_S01-S56.csv")

dataset = pd.read_csv('/content/drive/MyDrive/Iqram Sir/Fitbit_feature_merged_version_RepeatedData_S01-S56.csv')

<ipython-input-54-8de9d24b8596>:9: DtypeWarning: Columns (134,136,141,363,369,373,375,377,379,402,418,431,435,437,439,441,447,451) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('/content/drive/MyDrive/Iqram Sir/Fitbit_feature_merged_version_RepeatedData_S01-S56.csv')


In [37]:
dataset.head(10)

,Subject,Condition,Activity,Epoch Duration(min),Epoch,nni_counter,nni_mean,nni_min,nni_max,hr_mean,...,poms_wound_30d,poms_wound_30d_cd,poms_hematological_30d,poms_hematological_30d_cd,poms_pain_30d,poms_pain_30d_cd,case_report_form_complete,qc_yesno,qc_date,quality_check_complete
0,018-PDT,post,Extreme,10.0,1.0,93.0,568.74,517.0,689.0,105.83,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,1.0,45071.0,2.0
1,018-PDT,post,Heavy,10.0,1.0,69.0,841.03,789.0,923.0,71.44,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,1.0,45071.0,2.0
2,018-PDT,post,Heavy,10.0,2.0,81.0,613.40,550.0,740.0,98.15,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,1.0,45071.0,2.0
3,018-PDT,post,Light,10.0,1.0,87.0,772.76,681.0,882.0,77.99,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,1.0,45071.0,2.0
4,018-PDT,post,Light,10.0,10.0,76.0,826.87,789.0,895.0,72.61,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,1.0,45071.0,2.0
5,018-PDT,post,Light,10.0,100.0,83.0,941.77,869.0,1016.0,63.77,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,1.0,45071.0,2.0
6,018-PDT,post,Light,10.0,101.0,74.0,976.01,923.0,1034.0,61.53,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,1.0,45071.0,2.0
7,018-PDT,post,Light,10.0,102.0,64.0,949.97,895.0,1000.0,63.20,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,1.0,45071.0,2.0
8,018-PDT,post,Light,10.0,103.0,79.0,873.44,789.0,1052.0,68.99,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,1.0,45071.0,2.0
9,018-PDT,post,Light,10.0,104.0,68.0,995.13,923.0,1052.0,60.36,...,2.0,NaN,2.0,NaN,2.0,NaN,2.0,1.0,45071.0,2.0


In [38]:
#dataset[~dataset.isnull().any(axis=1)] # all rows containing non null values

In [39]:
#dataset['Distance Walked:'].isnull().sum()

**<font color='coral'>Adding VO2_Max_6MWT</font>**

Poor:  VO2max < 30<br>
Average: 30 < VO2max < 40<br>
Good: VO2max > 40<br>

In [ ]:
#target = "VO2_Max_6MWT"

#gender_dict = {"M":1,"F":2}
loc = dataset.columns.get_loc("5_min_HR_Recovery(stdev)") + 1
dataset.insert(loc,"VO2_Max_6MWT","")
for i in range(dataset.shape[0]):
  VO2max_temp =59.44 - 3.83 * dataset['gender'][i] - 0.56*(dataset["age_years"][i]+(dataset["age_months"][i])/12) - 0.48 * dataset["bmi"][i] + 0.04*0.3048*dataset["mwt_distance"][i]


  if VO2max_temp <= 30:
    dataset["VO2_Max_6MWT"][i] = "Poor"
  elif 30 < VO2max_temp <= 40:
    dataset["VO2_Max_6MWT"][i] = "Average"
  elif 40 < VO2max_temp:
    dataset["VO2_Max_6MWT"][i] = "Good"
  else:
    dataset["VO2_Max_6MWT"][i] = ""


In [56]:
dataset["VO2_Max_6MWT"].value_counts()

Poor    10424
           51
Name: VO2_Max_6MWT, dtype: int64

In [ ]:
#list(dataset.columns)

In [7]:
last_index = dataset.columns.get_loc("VO2_Max_6MWT")+1

dataset = dataset.iloc[:,:last_index]

**<font color='coral'>Null Dropping</font>**

In [8]:
dataset = dataset.dropna(axis=1,how='all') #column elemination
dataset = dataset.reset_index(drop=True)

In [9]:
dataset.shape

(10475, 121)

**<font color='coral'>Check Null data count per column</font>**

In [10]:
null_cols = dataset.columns[dataset.isnull().any()].tolist()

for i in null_cols:
  print(i,":",dataset[i].isna().sum().sum())
print('Total null: ',dataset.isna().sum().sum())

dfa_alpha1 : 1181
dfa_alpha2 : 1181
VO2_Max_6MWT : 51
Total null:  2413


**<font color='coral'>Filtering: <font color='blue'>Replacing null with mean if possible. (condition on `Subject, Condition, Activity`)</font>**

In [11]:
null_cols = dataset.columns[dataset.isnull().any()].tolist()

for current_col in null_cols:

  print(current_col)
  null_index = list(np.where(dataset[current_col].isna())[0])

  sub_con_act = []



  for i in null_index:
    if (dataset["Subject"][i],dataset["Condition"][i],dataset["Activity"][i]) not in sub_con_act:
      sub_con_act.append((dataset["Subject"][i],dataset["Condition"][i],dataset["Activity"][i]))


  for j in sub_con_act:
    temp_df = dataset.query("Condition == '"+j[1]+"' and Activity == '"+j[2]+"'  and Subject == '"+j[0]+"'")

    #calculating mean
    temp_mean = temp_df[current_col].mean()

    for i in list(temp_df.index):
      if str(dataset[current_col][i]) == 'nan':
        dataset[current_col][i] = temp_mean


dfa_alpha1


<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[current_col][i] = temp_mean
<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[current_col][i] = temp_mean
<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[current_col][i] = temp_mean
<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be 

dfa_alpha2


<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[current_col][i] = temp_mean
<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[current_col][i] = temp_mean
<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[current_col][i] = temp_mean
<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be 

VO2_Max_6MWT


<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[current_col][i] = temp_mean
<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[current_col][i] = temp_mean
<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[current_col][i] = temp_mean
<ipython-input-11-84fe668b7010>:25: SettingWithCopyWarning: 
A value is trying to be 

**<font color='coral'>Check Null data count per column after replacing</font>**

In [12]:
null_cols = dataset.columns[dataset.isnull().any()].tolist()

for i in null_cols:
  print(i,":",dataset[i].isna().sum().sum())
print('Total null: ',dataset.isna().sum().sum())

dfa_alpha1 : 5
dfa_alpha2 : 5
VO2_Max_6MWT : 51
Total null:  61


**removing rest of the null value**

In [13]:
dataset.dropna(axis=0, how='any', inplace = True) #row elemination
dataset = dataset.reset_index(drop=True)

In [14]:
dataset.shape

(10419, 121)



---



---



#Split Dataset into pre and post

In [15]:
pre_dataset = dataset.query("Condition == 'pre' and Activity == 'Light'").reset_index(drop=True)
post_dataset = dataset.query("Condition == 'post' and Activity == 'Light'").reset_index(drop=True)


In [16]:
pre_dataset.head(10)

,Subject,Condition,Activity,Epoch Duration(min),Epoch,nni_counter,nni_mean,nni_min,nni_max,hr_mean,...,1_min_HR_Recovery(stdev),2_min_HR_Recovery(mean),2_min_HR_Recovery(stdev),3_min_HR_Recovery(mean),3_min_HR_Recovery(stdev),4_min_HR_Recovery(mean),4_min_HR_Recovery(stdev),5_min_HR_Recovery(mean),5_min_HR_Recovery(stdev),VO2_Max_6MWT
0,018-PDT,pre,Light,10.0,1.0,93.0,702.12,600.0,810.0,85.98,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.14504
1,018-PDT,pre,Light,10.0,10.0,66.0,987.82,909.0,1052.0,60.80,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.14504
2,018-PDT,pre,Light,10.0,11.0,67.0,1012.39,952.0,1071.0,59.30,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.14504
3,018-PDT,pre,Light,10.0,12.0,66.0,967.62,833.0,1034.0,62.29,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.14504
4,018-PDT,pre,Light,10.0,13.0,71.0,1002.34,923.0,1200.0,59.98,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.14504
5,018-PDT,pre,Light,10.0,14.0,67.0,903.55,697.0,1016.0,67.43,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.14504
6,018-PDT,pre,Light,10.0,15.0,73.0,996.21,923.0,1132.0,60.29,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.14504
7,018-PDT,pre,Light,10.0,16.0,64.0,999.03,952.0,1132.0,60.12,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.14504
8,018-PDT,pre,Light,10.0,17.0,66.0,984.52,909.0,1034.0,60.99,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.14504
9,018-PDT,pre,Light,10.0,18.0,72.0,997.32,923.0,1090.0,60.28,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.14504


In [17]:
post_dataset.head(10)

,Subject,Condition,Activity,Epoch Duration(min),Epoch,nni_counter,nni_mean,nni_min,nni_max,hr_mean,...,1_min_HR_Recovery(stdev),2_min_HR_Recovery(mean),2_min_HR_Recovery(stdev),3_min_HR_Recovery(mean),3_min_HR_Recovery(stdev),4_min_HR_Recovery(mean),4_min_HR_Recovery(stdev),5_min_HR_Recovery(mean),5_min_HR_Recovery(stdev),VO2_Max_6MWT
0,018-PDT,post,Light,10.0,1.0,87.0,772.76,681.0,882.0,77.99,...,3.76,3.39,3.76,3.39,3.76,3.39,3.76,3.39,3.76,23.14504
1,018-PDT,post,Light,10.0,10.0,76.0,826.87,789.0,895.0,72.61,...,3.76,3.39,3.76,3.39,3.76,3.39,3.76,3.39,3.76,23.14504
2,018-PDT,post,Light,10.0,100.0,83.0,941.77,869.0,1016.0,63.77,...,3.76,3.39,3.76,3.39,3.76,3.39,3.76,3.39,3.76,23.14504
3,018-PDT,post,Light,10.0,101.0,74.0,976.01,923.0,1034.0,61.53,...,3.76,3.39,3.76,3.39,3.76,3.39,3.76,3.39,3.76,23.14504
4,018-PDT,post,Light,10.0,102.0,64.0,949.97,895.0,1000.0,63.20,...,3.76,3.39,3.76,3.39,3.76,3.39,3.76,3.39,3.76,23.14504
5,018-PDT,post,Light,10.0,103.0,79.0,873.44,789.0,1052.0,68.99,...,3.76,3.39,3.76,3.39,3.76,3.39,3.76,3.39,3.76,23.14504
6,018-PDT,post,Light,10.0,104.0,68.0,995.13,923.0,1052.0,60.36,...,3.76,3.39,3.76,3.39,3.76,3.39,3.76,3.39,3.76,23.14504
7,018-PDT,post,Light,10.0,105.0,70.0,929.97,857.0,1090.0,64.74,...,3.76,3.39,3.76,3.39,3.76,3.39,3.76,3.39,3.76,23.14504
8,018-PDT,post,Light,10.0,106.0,71.0,824.06,722.0,937.0,73.08,...,3.76,3.39,3.76,3.39,3.76,3.39,3.76,3.39,3.76,23.14504
9,018-PDT,post,Light,10.0,107.0,66.0,979.85,882.0,1034.0,61.31,...,3.76,3.39,3.76,3.39,3.76,3.39,3.76,3.39,3.76,23.14504


In [18]:
dataset = pre_dataset

In [19]:
dataset

,Subject,Condition,Activity,Epoch Duration(min),Epoch,nni_counter,nni_mean,nni_min,nni_max,hr_mean,...,1_min_HR_Recovery(stdev),2_min_HR_Recovery(mean),2_min_HR_Recovery(stdev),3_min_HR_Recovery(mean),3_min_HR_Recovery(stdev),4_min_HR_Recovery(mean),4_min_HR_Recovery(stdev),5_min_HR_Recovery(mean),5_min_HR_Recovery(stdev),VO2_Max_6MWT
0,018-PDT,pre,Light,10.0,1.0,93.0,702.12,600.0,810.0,85.98,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.145040
1,018-PDT,pre,Light,10.0,10.0,66.0,987.82,909.0,1052.0,60.80,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.145040
2,018-PDT,pre,Light,10.0,11.0,67.0,1012.39,952.0,1071.0,59.30,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.145040
3,018-PDT,pre,Light,10.0,12.0,66.0,967.62,833.0,1034.0,62.29,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.145040
4,018-PDT,pre,Light,10.0,13.0,71.0,1002.34,923.0,1200.0,59.98,...,3.24,3.93,3.24,3.93,3.24,3.93,3.24,3.93,3.24,23.145040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3058,001-JV,pre,Light,10.0,95.0,84.0,729.21,674.0,800.0,82.44,...,3.69,3.89,3.69,3.89,3.69,3.89,3.69,3.89,3.69,16.695781
3059,001-JV,pre,Light,10.0,96.0,75.0,727.93,705.0,750.0,82.46,...,3.69,3.89,3.69,3.89,3.69,3.89,3.69,3.89,3.69,16.695781
3060,001-JV,pre,Light,10.0,97.0,69.0,731.77,689.0,769.0,82.05,...,3.69,3.89,3.69,3.89,3.69,3.89,3.69,3.89,3.69,16.695781
3061,001-JV,pre,Light,10.0,98.0,83.0,778.73,731.0,810.0,77.09,...,3.69,3.89,3.69,3.89,3.69,3.89,3.69,3.89,3.69,16.695781




---


Encoding all str/obj columns

In [20]:
dataset = encode(dataset)

In [21]:
dataset.dtypes

Subject                       int64
Condition                     int64
Activity                      int64
Epoch Duration(min)         float64
Epoch                       float64
                             ...   
4_min_HR_Recovery(mean)     float64
4_min_HR_Recovery(stdev)    float64
5_min_HR_Recovery(mean)     float64
5_min_HR_Recovery(stdev)    float64
VO2_Max_6MWT                float64
Length: 121, dtype: object

Deleting all str/obj columns

In [22]:
#dataset = drop_str_col(dataset)



---



#Custom Spliting

In [23]:
len(set(dataset ["Subject"]))

47

In [25]:
num_of_train_sub = 28

train,test = sub_wise_splt(dataset,"Subject", num_of_train_sub)

In [26]:
train.shape

(1976, 121)

In [27]:
test.shape

(1087, 121)

#Custom Feature Selection

In [29]:

all_columns = list(dataset.columns)
new_keys=items_check_box(all_columns)

Checkbox(value=False, description='Subject', indent=False)

Checkbox(value=False, description='Condition', indent=False)

Checkbox(value=False, description='Activity', indent=False)

Checkbox(value=False, description='Epoch Duration(min)', indent=False)

Checkbox(value=False, description='Epoch', indent=False)

...

Checkbox(value=False, description='4_min_HR_Recovery(mean)', indent=False)

Checkbox(value=False, description='4_min_HR_Recovery(stdev)', indent=False)

Checkbox(value=False, description='5_min_HR_Recovery(mean)', indent=False)

Checkbox(value=False, description='5_min_HR_Recovery(stdev)', indent=False)

Checkbox(value=False, description='VO2_Max_6MWT', indent=False)

In [30]:
new_features = []
for i in range(len(new_keys)):
  if new_keys[i].value ==True:
    new_features.append(all_columns[i])

In [31]:
y_i = widgets.RadioButtons(
    options=all_columns,
    description='Select Y:',
    disabled=False
)
y_i

RadioButtons(description='Select Y:', options=('Subject', 'Condition', 'Activity', 'Epoch Duration(min)', 'Epo…

In [32]:
X = dataset.loc[:,new_features]
y = dataset[y_i.value]

In [33]:
X_train = train.loc[:,new_features]
y_train = train[y_i.value]

X_test = test.loc[:,new_features]
y_test = test[y_i.value]



---



---



#Feture Selection Kbest

In [ ]:
"""
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
"""

In [ ]:
"""
column_name = 'DASI - VO2 peak score:'
column_index = dataset.columns.get_loc(column_name)
column_index
"""

184

In [ ]:
"""
from sklearn.feature_selection import SelectKBest
bestfeatures = SelectKBest(k=20)
fit = bestfeatures.fit(X,y)


dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']


number_of_feat=int(input("Number Of Best Features: "))


imp=featureScores.nlargest(number_of_feat,'Score')
print(imp)


list_of_feat=[]

for i in range(number_of_feat):
  list_of_feat.append(imp.iloc[:,0:1].values[i][0])


X_new = dataset[list_of_feat]
y_new = dataset.iloc[:, 184]
"""

In [ ]:
"""X_new.shape"""

(1354, 30)

In [ ]:
"""
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
"""



---



#Models

#ADA Boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_defult = AdaBoostClassifier(random_state=0)
ada_defult.fit(X_train, y_train)
y_pred = ada_defult.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(ada_defult,1,'AdaBoostClassifier')]=accuracy_score(y_test, y_pred)


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
N=200
k_range = range (1,N+1)
scores={}
scores_list = []
for k in k_range:
  classifier = AdaBoostClassifier(n_estimators=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt
#plot the relationship between K and the testing accuracy
plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best n_estimators:")
best_estimator=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_estimator)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_best_estimator = AdaBoostClassifier(n_estimators=best_estimator,random_state=0)
ada_best_estimator.fit(X_train, y_train)
y_pred = ada_best_estimator.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(ada_best_estimator,1,'AdaBoostClassifier')]=accuracy_score(y_test, y_pred)

#Graddient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

gradBoost_default = GradientBoostingClassifier(random_state=0)
gradBoost_default.fit(X_train, y_train)
y_pred = gradBoost_default.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(gradBoost_default,2,'GradientBoostingClassifier')]=accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
N=150
k_range = range (1,N+1)
scores={}
scores_list = []
for k in k_range:
  classifier = GradientBoostingClassifier(n_estimators=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best n_estimators:")
best_estimator=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_estimator)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
N=30
k_range = range (1,N+1)
scores={}
scores_list = []
for k in k_range:
  classifier = GradientBoostingClassifier(max_depth=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best Depth:")
best_depth=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_depth)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradBoost_estimator = GradientBoostingClassifier(n_estimators=best_estimator,random_state=0)
gradBoost_estimator.fit(X_train, y_train)
y_pred = gradBoost_estimator.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(gradBoost_estimator,2,'GradientBoostingClassifier')]=accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradBoost_depth = GradientBoostingClassifier(max_depth=best_depth,random_state=0)
gradBoost_depth.fit(X_train, y_train)
y_pred = gradBoost_depth.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

result[(gradBoost_depth,2,'GradientBoostingClassifier')]=accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradBoost_all = GradientBoostingClassifier(n_estimators=best_estimator,max_depth=best_depth,random_state=0)
gradBoost_all.fit(X_train, y_train)
y_pred = gradBoost_all.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

result[(gradBoost_all,2,'GradientBoostingClassifier')]=accuracy_score(y_test, y_pred)

#Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_default = RandomForestClassifier(random_state=0)
rf_default.fit(X_train, y_train)
y_pred=rf_default.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(rf_default,3,'RandomForestClassifier')]=accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
N=150
k_range = range (1,N+1)
scores={}
scores_list = []
for k in k_range:
  classifier = RandomForestClassifier(n_estimators=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best n_estimators:")
best_estimator=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_estimator)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
N=150
k_range = range (1,N+1)
scores={}
scores_list = []
for k in k_range:
  classifier = RandomForestClassifier(max_depth=k,random_state=0)
  classifier.fit(X_train, y_train)
  y_pred=classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))



%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best Depth:")
best_depth=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_depth)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_estimator = RandomForestClassifier(n_estimators=best_estimator,random_state=0)
rf_estimator.fit(X_train, y_train)
y_pred=rf_estimator.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(rf_estimator,3,'RandomForestClassifier')]=accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_depth = RandomForestClassifier(max_depth=best_depth,random_state=0)
rf_depth.fit(X_train, y_train)
y_pred=rf_depth.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(rf_depth,3,'RandomForestClassifier')]=accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_all = RandomForestClassifier(n_estimators=best_estimator,max_depth=best_depth,random_state=0)
rf_all.fit(X_train, y_train)
y_pred=rf_all.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(rf_all,3,'RandomForestClassifier')]=accuracy_score(y_test, y_pred)

#XGB

In [ ]:


import xgboost as xgb
xgb_deafult = xgb.XGBClassifier(random_state=0)
xgb_deafult.fit(X_train,y_train)
y_pred = xgb_deafult.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(xgb_deafult,4,'xgboost')]=accuracy_score(y_test, y_pred)

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
N=250
k_range = range (1,N+1)
scores={}
scores_list = []
for k in k_range:
  xgb_classifier = xgb.XGBClassifier(n_estimators=k,random_state=0)
  xgb_classifier.fit(X_train, y_train)
  y_pred=xgb_classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))


%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best n_estimators:")
best_estimator=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_estimator)

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
N=250
k_range = range (1,N+1)
scores={}
scores_list = []
for k in k_range:
  xgb_classifier = xgb.XGBClassifier(max_depth=k,random_state=0)
  xgb_classifier.fit(X_train, y_train)
  y_pred=xgb_classifier.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(N)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))


%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best depth:")
best_depth=list(scores.keys())[scores_list.index(max(scores_list))]
print(best_depth)

In [ ]:
import xgboost as xgb
xgb_depth = xgb.XGBClassifier(max_depth=best_depth,random_state=0)
xgb_depth.fit(X_train,y_train)
y_pred = xgb_depth.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(xgb_depth,4,'xgboost')]=accuracy_score(y_test, y_pred)
print(xgb_depth)

In [ ]:
import xgboost as xgb
xgb_estimator = xgb.XGBClassifier(n_estimators=best_estimator,random_state=0)
xgb_estimator.fit(X_train,y_train)
y_pred = xgb_estimator.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(xgb_estimator,4,'xgboost')]=accuracy_score(y_test, y_pred)
print(xgb_estimator)

In [ ]:
import xgboost as xgb
xgb_all = xgb.XGBClassifier(n_estimators=best_estimator,max_depth=best_depth,random_state=0)
xgb_all.fit(X_train,y_train)
y_pred = xgb_all.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(xgb_all,4,'xgboost')]=accuracy_score(y_test, y_pred)

#KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_default = KNeighborsClassifier()
knn_default.fit(X_train, y_train)
y_pred=knn_default.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(knn_default,5,'KNeighborsClassifier')]=accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
Neighbors=105
k_range = range (1,Neighbors+1)
scores={}
scores_list = []
for k in k_range:
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)
  y_pred=knn.predict(X_test)
  scores[k] = accuracy_score(y_test,y_pred)
  scores_list.append(accuracy_score(y_test,y_pred))
  print(str(k)+"/"+str(Neighbors)+" round completed......................... Accurecy: "+str(accuracy_score(y_test,y_pred)))


%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize = (25,10))
plt.plot(k_range,scores_list)
plt.xlabel('Value of n_estimators')
plt.ylabel ('Testing Accuracy')



print("The best Depth:")
best=list(scores.keys())[scores_list.index(max(scores_list))]
print(best)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_neighbors = KNeighborsClassifier(n_neighbors=best)
knn_neighbors.fit(X_train, y_train)
y_pred=knn_neighbors.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(knn_neighbors,5,'KNeighborsClassifier')]=accuracy_score(y_test, y_pred)

#NB

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_deafult = GaussianNB()
nb_deafult.fit(X_train, y_train)
y_pred = nb_deafult.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))
result[(nb_deafult,6,'GaussianNB')]=accuracy_score(y_test, y_pred)

#Result

In [ ]:

models=[]

for i in result:
  models.append(i[0])
  print(i[0],i[1]," : ",result[i])
  print("---------------------------------------------------------------")
  print()


In [ ]:
sorted_list=[]
sorted_list = sorted(result, key=result.get,reverse=True)

for i in sorted_list:
  print(i,"  : ",result[i])
  print("-------------------------------------------------------------------------------------------------")

print(sorted_list)


flage=[]
best_models=[]
it=0

for i in sorted_list:
  if it==4:
    break

  if i[1] not in flage:
    best_models.append((i[0],i[2]))
    flage.append(i[1])
    it+=1


In [ ]:
print("best_models:")
for i in best_models:
  print(i)


#Testing Accuracy For Best 4 Models

In [ ]:

for i in best_models:
  print("--------------------------------------------------")
  print(i[0])
  y_pred=i[0].predict(X_train)
  print(confusion_matrix(y_train, y_pred))
  print(classification_report(y_train,y_pred))
  print("Accurecy: ",accuracy_score(y_train, y_pred))

#Confusion Matrix For Testing

In [ ]:
new_keys_6=models_check_box(models)

In [ ]:

from yellowbrick.classifier import ConfusionMatrix


for i in range(len(new_keys_6)):
  if new_keys_6[i].value ==True:
    cm = ConfusionMatrix(models[i], classes=classes)

    cm.fit(X_train, y_train)
    cm.score(X_test, y_test)
    cm.show()

#Confusion Matrix For Training

In [ ]:
new_keys_5=models_check_box(models)

In [ ]:

for i in range(len(new_keys_5)):
  if new_keys_5[i].value ==True:
    cm = ConfusionMatrix(models[i], classes=classes)

    cm.fit(X_train, y_train)
    cm.score(X_train, y_train)
    cm.show()
    print("-------------------------------")
    print("-------------------------------")

#Result CSV<br>
Predicted and Actual Row will be added at the end index

In [ ]:
def reverse_transfrom(my_list):
  for i in range(len(my_list)):
    if my_list[i] == 0:
      my_list[i] = 'Control'
    else:
      my_list[i] = 'Patient'
  return my_list

In [ ]:
new_keys_4=models_check_box(models)

In [ ]:
for i in range(len(new_keys_4)):
  if new_keys_4[i].value ==True:
    temp=X_test_with_subject

    prediction_cls = reverse_transfrom(list(models[i].predict(X_test[0:])))
    actual_cls = reverse_transfrom(list(y_test[0:]))

    temp["Actual"] = actual_cls
    temp["Predicted"] = prediction_cls

    temp.to_csv(str(models[i])+".csv")